## Key Opinion Leaders. Top 100 ERC20 Holders by Tokens

In [4]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML, display
import pandas as pd

from config import PROJECT_ID, \
    ERC20_ANALYSIS_DATASET_NAME, ERC20_BALANCES_TABLE_NAME, \
    ERC20_TOP_DISTRIBUTION_TABLE_NAME
from utils import drop_table, create_table

DROP_TABLES = True
CREATE_TABLES = True

### Distribution Rule

<table style="text-align: center">
    <thead style="text-align: center">
        <tr>
            <th rowspan=2></th>
            <th colspan=3>Grade</th>
        </tr>
        <tr>
            <th>1</th>
            <th>2</th>
            <th>3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: left"> Top 100 ERC20 Holders by Tokens </td>
            <td> - </td>
            <td> - </td>
            <td> only hear </td>
        </tr>
    </tbody>
</table>

In [5]:
number_top_address = 100

### Create Distribution Table

In [6]:
query_1 = f'''
    SELECT
        address,
        3 AS grade,
        token_address,
        eth_balance,
        rank
    FROM (
        SELECT
            address,
            token_address,
            eth_balance,
            RANK() OVER (PARTITION BY token_address ORDER BY eth_balance DESC) AS rank
        FROM `{PROJECT_ID}.{ERC20_ANALYSIS_DATASET_NAME}.{ERC20_BALANCES_TABLE_NAME}`
        WHERE balance > 0
        )
    WHERE rank <= {number_top_address}
'''

if DROP_TABLES:
    drop_table(ERC20_TOP_DISTRIBUTION_TABLE_NAME, dataset_name=ERC20_ANALYSIS_DATASET_NAME)

if CREATE_TABLES:
    create_table(query_1, ERC20_TOP_DISTRIBUTION_TABLE_NAME, dataset_name=ERC20_ANALYSIS_DATASET_NAME)

Table erc20_analysis:erc20_top_distribution has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/erc20_analysis/tables/erc20_top_distribution?prettyPrint=false: Not found: Table cosmic-keep-223223:erc20_analysis.erc20_top_distribution
Table erc20_analysis:erc20_top_distribution has been created and filled 3528 rows.
